In [ ]:
import pandas as pd
df_clean = pd.read_csv("base_final.csv", sep=";")
print(df_clean.columns.tolist())

In [ ]:
print(df_clean.head()) 

In [ ]:
import mysql.connector
import pandas as pd
from pymongo import MongoClient

# Conectar a MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',
    database='repositorio_final2'
)

df_clean = pd.read_sql("SELECT * FROM datos_completos", conn)

# Conectar a MongoDB
cliente_mongo = MongoClient("mongodb://localhost:27017/")
db = cliente_mongo["proyecto_datos"]
coleccion = db["datos_integrados"]

# Exportar los datos
coleccion.insert_many(df_clean.to_dict(orient="records"))

print(" Exportado de MySQL a MongoDB ")

In [ ]:

import mysql.connector
# Conectar a MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',
    database='repositorio_final2'
)
cursor = conn.cursor()

# Crear tabla si no existe
cursor.execute('''
CREATE TABLE IF NOT EXISTS datos_completos (
    id INT, fecha VARCHAR(50), horas_sueno DOUBLE, minutos_meditacion DOUBLE,
    minutos_ejercicio DOUBLE, tipo_desayuno VARCHAR(100), journaling VARCHAR(10),
    puntaje_productividad DOUBLE, estado_animo VARCHAR(100), notas VARCHAR(255),
    fecha_transaccion VARCHAR(50), edad_cliente DOUBLE, ubicacion_cliente VARCHAR(100),
    cantidad DOUBLE, precio_unitario DOUBLE
);
''')

# Insertar primeros 10 registros
for _, fila in df_clean.iterrows():
    cursor.execute('''
        INSERT INTO datos_completos VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    ''', tuple(fila[col] if pd.notna(fila[col]) else None for col in [
        'id', 'fecha', 'horas_sueno', 'minutos_meditacion', 'minutos_ejercicio',
        'tipo_desayuno', 'journaling', 'puntaje_productividad', 'estado_animo',
        'notas', 'fecha_transaccion', 'edad_cliente', 'ubicacion_cliente',
        'cantidad', 'precio_unitario'
    ]))

# Confirmar y cerrar
conn.commit()
conn.close()

print("Se crearon la tabla y se insertaron 10 registros correctamente.")
